# Лабораторная работа 5

   по теме
                 
   **Распознавание объектов на фотографиях**
   
   ****

   Выполнил студент

   Группы БСТ1801

   Харатишвили Заза

In [3]:
#Импорты
from keras.datasets import cifar10
from keras.models import Model
from keras.layers import Input, Convolution2D, MaxPooling2D, Dense, Dropout, Flatten

from tensorflow.keras.utils import to_categorical
import numpy as np

from keras.utils import np_utils

In [4]:
batch_size = 32 # на каждой итерации одновременно обрабатываем 32 обучающих образца
num_epochs = 100 # количество итераций
kernel_size = 3 # размер ядра в свертывающих слоях
pool_size = 2 # размер подвыборки в слоях подвыборки
conv_depth_1 = 32 # количество ядер в свертывающих слоях
conv_depth_2 = 64 # увеличиваем в два раза после слоя подвыборки 
drop_prob_1 = 0.25 # убираем нейроны после слоя подвыборки с вероятностью 0.25
drop_prob_2 = 0.5 # убираем нейроны после полносвязного слоя с вероятностью 0.5
hidden_size = 512 # количество нейронов в полносвязном слое

In [5]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data() # получаем данные CIFAR-10 
#96x96,255
# 50000 данных для обучения, 10000 для тестирования, 10 классов картинок
num_train, depth, height, width = X_train.shape
num_test = X_test.shape[0]
num_classes = np.unique(y_train).shape[0]

# Приведем к виду с плавающей точкой
X_train = X_train.astype('float32') 
X_test = X_test.astype('float32')
# Преобразуем интетнсивность пикселей до [0 ,1]
X_train /= np.max(X_train)
X_test /= np.max(X_train) #делим на обучающее, тк нельзя показывать алгоритмам тестовые данные до окончания 
# процесса обучения

# переводим в унитарный код
Y_train = to_categorical(y_train, num_classes)
Y_test = to_categorical(y_test, num_classes)

In [6]:
inp = Input(shape=(depth, height, width))
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
conv_1 = Convolution2D(conv_depth_1, (kernel_size, kernel_size), padding='same', activation='relu')(inp) #Сверточный слой
conv_2 = Convolution2D(conv_depth_1, (kernel_size, kernel_size), padding='same', activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size), padding='same')(conv_2) # Слой подвыборки
drop_1 = Dropout(drop_prob_1)(pool_1)
# Conv [64] -> Conv [64] -> Pool (with dropout on the pooling layer)
conv_3 = Convolution2D(conv_depth_2, (kernel_size, kernel_size), padding='same', activation='relu')(drop_1)
conv_4 = Convolution2D(conv_depth_2, (kernel_size, kernel_size), padding='same', activation='relu')(conv_3)
pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size), padding='same')(conv_4)
drop_2 = Dropout(drop_prob_1)(pool_2)
# Now flatten to 1D, apply FC -> ReLU (with dropout) -> softmax
flat = Flatten()(drop_2) # Трансформируем в одномерный вектор
hidden = Dense(hidden_size, activation='relu')(flat)
drop_3 = Dropout(drop_prob_2)(hidden)
out = Dense(num_classes, activation='softmax')(drop_3)

model = Model(inp, out) # Укажем слой входа и слой выхода

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, Y_train, batch_size=batch_size, epochs=num_epochs,
          verbose=1, validation_split=0.1) # Оставляем 10 процентов для тестирования
# Оценим модель
model.evaluate(X_test, Y_test, verbose=1) # Оценим обученную модель на тестовых данных

Epoch 1/100
1407/1407 [==============================] - 53s 37ms/step - loss: 1.5683 - accuracy: 0.4249 - val_loss: 1.1585 - val_accuracy: 0.5794
Epoch 2/100
1407/1407 [==============================] - 52s 37ms/step - loss: 1.1393 - accuracy: 0.5930 - val_loss: 0.9520 - val_accuracy: 0.6650
Epoch 3/100
1407/1407 [==============================] - 52s 37ms/step - loss: 0.9900 - accuracy: 0.6473 - val_loss: 0.8460 - val_accuracy: 0.7052
Epoch 4/100
1407/1407 [==============================] - 52s 37ms/step - loss: 0.8979 - accuracy: 0.6816 - val_loss: 0.8068 - val_accuracy: 0.7170
Epoch 5/100
1407/1407 [==============================] - 52s 37ms/step - loss: 0.8264 - accuracy: 0.7075 - val_loss: 0.7663 - val_accuracy: 0.7362
Epoch 6/100
1407/1407 [==============================] - 52s 37ms/step - loss: 0.7673 - accuracy: 0.7287 - val_loss: 0.7339 - val_accuracy: 0.7498
Epoch 7/100
1407/1407 [==============================] - 52s 37ms/step - loss: 0.7225 - accuracy: 0.7439 - val_loss: 0

1407/1407 [==============================] - 52s 37ms/step - loss: 0.2872 - accuracy: 0.9011 - val_loss: 0.7482 - val_accuracy: 0.8000
Epoch 57/100
1407/1407 [==============================] - 52s 37ms/step - loss: 0.2807 - accuracy: 0.9060 - val_loss: 0.7556 - val_accuracy: 0.7980
Epoch 58/100
1407/1407 [==============================] - 52s 37ms/step - loss: 0.2832 - accuracy: 0.9051 - val_loss: 0.8149 - val_accuracy: 0.7904
Epoch 59/100
1407/1407 [==============================] - 52s 37ms/step - loss: 0.2869 - accuracy: 0.9027 - val_loss: 0.8199 - val_accuracy: 0.7878
Epoch 60/100
1407/1407 [==============================] - 52s 37ms/step - loss: 0.2808 - accuracy: 0.9054 - val_loss: 0.7630 - val_accuracy: 0.7932
Epoch 61/100
1407/1407 [==============================] - 52s 37ms/step - loss: 0.2770 - accuracy: 0.9068 - val_loss: 0.8016 - val_accuracy: 0.7778
Epoch 62/100
1407/1407 [==============================] - 52s 37ms/step - loss: 0.2761 - accuracy: 0.9060 - val_loss: 0.8043 

[382.1963806152344, 0.5044999718666077]

Исследуем сеть без слоя Dropout

In [7]:
inp = Input(shape=(depth, height, width))
# Conv [32] -> Conv [32] -> Pool
conv_1 = Convolution2D(conv_depth_1, (kernel_size, kernel_size), padding='same', activation='relu')(inp)
conv_2 = Convolution2D(conv_depth_1, (kernel_size, kernel_size), padding='same', activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size), padding='same')(conv_2)

# Conv [64] -> Conv [64] -> Pool
conv_3 = Convolution2D(conv_depth_2, (kernel_size, kernel_size), padding='same', activation='relu')(pool_1)
conv_4 = Convolution2D(conv_depth_2, (kernel_size, kernel_size), padding='same', activation='relu')(conv_3)
pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size), padding='same')(conv_4)

# Now flatten to 1D, apply FC -> ReLU -> softmax
flat = Flatten()(conv_4) # Трансформируем в одномерный вектор
hidden = Dense(hidden_size, activation='relu')(flat)
drop_3 = Dropout(drop_prob_2)(hidden)
out = Dense(num_classes, activation='softmax')(drop_3)

model = Model(inp, out) # Укажем слой входа и слой выхода

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, Y_train, batch_size=batch_size, epochs=num_epochs,
          verbose=1, validation_split=0.1) # Оставляем 10 процентов для тестирования

model.evaluate(X_test, Y_test, verbose=1) # Оценим обученную модель на тестовых данных

Epoch 1/100
1407/1407 [==============================] - 72s 51ms/step - loss: 1.4815 - accuracy: 0.4624 - val_loss: 1.0791 - val_accuracy: 0.6204
Epoch 2/100
1407/1407 [==============================] - 72s 51ms/step - loss: 1.0718 - accuracy: 0.6187 - val_loss: 0.9181 - val_accuracy: 0.6696
Epoch 3/100
1407/1407 [==============================] - 72s 51ms/step - loss: 0.8926 - accuracy: 0.6863 - val_loss: 0.8144 - val_accuracy: 0.7218
Epoch 4/100
1407/1407 [==============================] - 72s 51ms/step - loss: 0.7479 - accuracy: 0.7365 - val_loss: 0.7690 - val_accuracy: 0.7308
Epoch 5/100
1407/1407 [==============================] - 73s 52ms/step - loss: 0.6243 - accuracy: 0.7797 - val_loss: 0.7633 - val_accuracy: 0.7436
Epoch 6/100
1407/1407 [==============================] - 72s 51ms/step - loss: 0.5111 - accuracy: 0.8194 - val_loss: 0.8104 - val_accuracy: 0.7388
Epoch 7/100
1407/1407 [==============================] - 72s 51ms/step - loss: 0.4133 - accuracy: 0.8549 - val_loss: 0

1407/1407 [==============================] - 73s 52ms/step - loss: 0.0895 - accuracy: 0.9748 - val_loss: 2.1292 - val_accuracy: 0.7290
Epoch 57/100
1407/1407 [==============================] - 73s 52ms/step - loss: 0.0818 - accuracy: 0.9763 - val_loss: 2.3687 - val_accuracy: 0.7302
Epoch 58/100
1407/1407 [==============================] - 73s 52ms/step - loss: 0.0868 - accuracy: 0.9752 - val_loss: 2.0340 - val_accuracy: 0.7334
Epoch 59/100
1407/1407 [==============================] - 73s 52ms/step - loss: 0.0898 - accuracy: 0.9758 - val_loss: 2.0638 - val_accuracy: 0.7336
Epoch 60/100
1407/1407 [==============================] - 73s 52ms/step - loss: 0.0911 - accuracy: 0.9743 - val_loss: 2.3081 - val_accuracy: 0.7342
Epoch 61/100
1407/1407 [==============================] - 73s 52ms/step - loss: 0.0775 - accuracy: 0.9775 - val_loss: 2.2909 - val_accuracy: 0.7314
Epoch 62/100
1407/1407 [==============================] - 73s 52ms/step - loss: 0.0884 - accuracy: 0.9750 - val_loss: 2.0016 

[839.2357788085938, 0.4675999879837036]

Уменьшим размер ядра свертки

In [8]:
kernel_size = 1 # размер ядра в свертывающих слоях

inp = Input(shape=(depth, height, width))
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
conv_1 = Convolution2D(conv_depth_1, (kernel_size, kernel_size), padding='same', activation='relu')(inp)
conv_2 = Convolution2D(conv_depth_1, (kernel_size, kernel_size), padding='same', activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size), padding='same')(conv_2)
drop_1 = Dropout(drop_prob_1)(pool_1)
# Conv [64] -> Conv [64] -> Pool (with dropout on the pooling layer)
conv_3 = Convolution2D(conv_depth_2, (kernel_size, kernel_size), padding='same', activation='relu')(drop_1)
conv_4 = Convolution2D(conv_depth_2, (kernel_size, kernel_size), padding='same', activation='relu')(conv_3)
pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size), padding='same')(conv_4)
drop_2 = Dropout(drop_prob_1)(pool_2)
# Now flatten to 1D, apply FC -> ReLU (with dropout) -> softmax
flat = Flatten()(drop_2) # Трансформируем в одномерный вектор
hidden = Dense(hidden_size, activation='relu')(flat)
drop_3 = Dropout(drop_prob_2)(hidden)
out = Dense(num_classes, activation='softmax')(drop_3)

model = Model(inp, out) # Укажем слой входа и слой выхода

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, Y_train, batch_size=batch_size, epochs=num_epochs,
          verbose=1, validation_split=0.1) # Оставляем 10 процентов для тестирования

model.evaluate(X_test, Y_test, verbose=1) # Оценим обученную модель на тестовых данных

Epoch 1/100
1407/1407 [==============================] - 23s 16ms/step - loss: 1.6956 - accuracy: 0.3848 - val_loss: 1.5026 - val_accuracy: 0.4716
Epoch 2/100
1407/1407 [==============================] - 23s 16ms/step - loss: 1.4595 - accuracy: 0.4755 - val_loss: 1.3295 - val_accuracy: 0.5256
Epoch 3/100
1407/1407 [==============================] - 23s 17ms/step - loss: 1.3873 - accuracy: 0.5007 - val_loss: 1.3045 - val_accuracy: 0.5364
Epoch 4/100
1407/1407 [==============================] - 23s 16ms/step - loss: 1.3323 - accuracy: 0.5240 - val_loss: 1.2554 - val_accuracy: 0.5570
Epoch 5/100
1407/1407 [==============================] - 23s 17ms/step - loss: 1.2919 - accuracy: 0.5385 - val_loss: 1.2222 - val_accuracy: 0.5772
Epoch 6/100
1407/1407 [==============================] - 23s 17ms/step - loss: 1.2617 - accuracy: 0.5506 - val_loss: 1.2142 - val_accuracy: 0.5720
Epoch 7/100
1407/1407 [==============================] - 23s 16ms/step - loss: 1.2246 - accuracy: 0.5671 - val_loss: 1

1407/1407 [==============================] - 25s 18ms/step - loss: 0.6048 - accuracy: 0.7823 - val_loss: 1.2819 - val_accuracy: 0.6100
Epoch 57/100
1407/1407 [==============================] - 24s 17ms/step - loss: 0.5950 - accuracy: 0.7837 - val_loss: 1.3249 - val_accuracy: 0.5952
Epoch 58/100
1407/1407 [==============================] - 23s 16ms/step - loss: 0.5906 - accuracy: 0.7864 - val_loss: 1.3023 - val_accuracy: 0.6060
Epoch 59/100
1407/1407 [==============================] - 23s 16ms/step - loss: 0.5817 - accuracy: 0.7887 - val_loss: 1.2957 - val_accuracy: 0.6034
Epoch 60/100
1407/1407 [==============================] - 23s 16ms/step - loss: 0.5768 - accuracy: 0.7885 - val_loss: 1.2687 - val_accuracy: 0.6120
Epoch 61/100
1407/1407 [==============================] - 23s 16ms/step - loss: 0.5715 - accuracy: 0.7894 - val_loss: 1.2848 - val_accuracy: 0.6174
Epoch 62/100
1407/1407 [==============================] - 23s 16ms/step - loss: 0.5718 - accuracy: 0.7923 - val_loss: 1.3302 

[2702.110107421875, 0.10920000076293945]

Увеличим размер ядра свертки

In [9]:
kernel_size = 4 # размер ядра в свертывающих слоях

inp = Input(shape=(depth, height, width))
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
conv_1 = Convolution2D(conv_depth_1, (kernel_size, kernel_size), padding='same', activation='relu')(inp)
conv_2 = Convolution2D(conv_depth_1, (kernel_size, kernel_size), padding='same', activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size), padding='same')(conv_2)
drop_1 = Dropout(drop_prob_1)(pool_1)
# Conv [64] -> Conv [64] -> Pool (with dropout on the pooling layer)
conv_3 = Convolution2D(conv_depth_2, (kernel_size, kernel_size), padding='same', activation='relu')(drop_1)
conv_4 = Convolution2D(conv_depth_2, (kernel_size, kernel_size), padding='same', activation='relu')(conv_3)
pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size), padding='same')(conv_4)
drop_2 = Dropout(drop_prob_1)(pool_2)
# Now flatten to 1D, apply FC -> ReLU (with dropout) -> softmax
flat = Flatten()(drop_2) # Трансформируем в одномерный вектор
hidden = Dense(hidden_size, activation='relu')(flat)
drop_3 = Dropout(drop_prob_2)(hidden)
out = Dense(num_classes, activation='softmax')(drop_3)

model = Model(inp, out) # Укажем слой входа и слой выхода

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, Y_train, batch_size=batch_size, epochs=num_epochs,
          verbose=1, validation_split=0.1) # Оставляем 10 процентов для тестирования

model.evaluate(X_test, Y_test, verbose=1) # Оценим обученную модель на тестовых данных

Epoch 1/100
1407/1407 [==============================] - 76s 54ms/step - loss: 1.5693 - accuracy: 0.4301 - val_loss: 1.1885 - val_accuracy: 0.5686
Epoch 2/100
1407/1407 [==============================] - 75s 54ms/step - loss: 1.1557 - accuracy: 0.5879 - val_loss: 0.9484 - val_accuracy: 0.6688
Epoch 3/100
1407/1407 [==============================] - 75s 53ms/step - loss: 0.9920 - accuracy: 0.6475 - val_loss: 0.8591 - val_accuracy: 0.7010
Epoch 4/100
1407/1407 [==============================] - 75s 53ms/step - loss: 0.8956 - accuracy: 0.6855 - val_loss: 0.8086 - val_accuracy: 0.7134
Epoch 5/100
1407/1407 [==============================] - 75s 53ms/step - loss: 0.8300 - accuracy: 0.7089 - val_loss: 0.7367 - val_accuracy: 0.7412
Epoch 6/100
1407/1407 [==============================] - 75s 54ms/step - loss: 0.7737 - accuracy: 0.7275 - val_loss: 0.7590 - val_accuracy: 0.7410
Epoch 7/100
1407/1407 [==============================] - 76s 54ms/step - loss: 0.7363 - accuracy: 0.7433 - val_loss: 0

1407/1407 [==============================] - 75s 54ms/step - loss: 0.3343 - accuracy: 0.8856 - val_loss: 0.8218 - val_accuracy: 0.7894
Epoch 57/100
1407/1407 [==============================] - 75s 53ms/step - loss: 0.3328 - accuracy: 0.8879 - val_loss: 0.8341 - val_accuracy: 0.7842
Epoch 58/100
1407/1407 [==============================] - 75s 53ms/step - loss: 0.3391 - accuracy: 0.8849 - val_loss: 0.7798 - val_accuracy: 0.7802
Epoch 59/100
1407/1407 [==============================] - 75s 53ms/step - loss: 0.3353 - accuracy: 0.8885 - val_loss: 0.8454 - val_accuracy: 0.7836
Epoch 60/100
1407/1407 [==============================] - 75s 54ms/step - loss: 0.3382 - accuracy: 0.8872 - val_loss: 0.8298 - val_accuracy: 0.7870
Epoch 61/100
1407/1407 [==============================] - 76s 54ms/step - loss: 0.3362 - accuracy: 0.8876 - val_loss: 0.8381 - val_accuracy: 0.7790
Epoch 62/100
1407/1407 [==============================] - 75s 53ms/step - loss: 0.3299 - accuracy: 0.8887 - val_loss: 0.8531 

[481.2984619140625, 0.5414000153541565]

## Вывод

В результате выполнения лабораторной работы можно сделать вывод о том, что от параметра DropOut напрямую зависит результат обучения модели, если указать слишком большую вероятность отключения нейрона, модель может неккоректно обучиться. При уменьшении размера ядра заметно ухудшается точность при оценке модели, а также увеличиваются потери и уменьшается точность обучения, но заметно уменьшается время компиляции. А при увеличении размера, наоборот, точность увеличивается, как и время компиляции, а потери уменьшаются.